In [1]:
import numpy as np
from astropy.modeling import models, fitting
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.optimize import curve_fit
import math
import os
from scipy.interpolate import interp2d
from scipy import ndimage, misc

In [ ]:
def tand(x):
    return math.tan(x * math.pi / 180)

def sind(x):
    return math.sin(x * math.pi / 180)

def cosd(x):
    return math.cos(x * math.pi / 180)

In [ ]:
def converting(img,PA=0,inclination=0,final_radius=100,pw=360,r0=40,times=2,center=None):    
    theta_ , R_ = np.meshgrid(np.linspace(0, 2*np.pi, pw), 
                            np.arange(0, final_radius))
    if center=None:
        center=[len(img)/2,len(img)/2]
    
    x=np.linspace(0, len(img)-1,len(img), dtype=int)
    y=np.linspace(0, len(img)-1,len(img), dtype=int)
    f=interp2d(x,y,img,kind='cubic')
    z2=[[[]for i in range(pw)]for i in range(final_radius)]
    theta=0
    while theta<90:
        r=1
        while r<final_radius:
            rr=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)
            x=rr*cosd(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+PA)+center[0]
            y=rr*sind(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+PA)+center[1]
            z2[r][theta]=float(f(x,y))*((rr/r0)**times)

            r+=1
        theta+=1
    theta=90
    r=1
    while r<final_radius:
        rr=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)
        x=(r*(cosd(inclination))*cosd(90+PA))+center[0]
        y=(r*(cosd(inclination))*sind(90+PA))+center[1]
        z2[r][theta]=float(f(x,y))*((rr/r0)**times)
        r+=1


    theta=91
    while theta<=180:
        r=1
        while r<final_radius:
            rr=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)
            x=rr*cosd(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+180+PA)+center[0]
            y=rr*sind(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+180+PA)+center[1]
            z2[r][theta]=float(f(x,y))*((rr/r0)**times)

            r+=1
        theta+=1

    while theta<270:
        r=1

        while r<final_radius:
            rr=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)
            x=rr*cosd(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+180+PA)+center[0]
            y=rr*sind(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+180+PA)+center[1]
            z2[r][theta]=float(f(x,y))*((rr/r0)**times)

            r+=1
        theta+=1

    theta=270
    r=1
    while r<final_radius:
        rr=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)
        x=-(r*(cosd(inclination))*cosd(90+PA))+center[0]
        y=-(r*(cosd(inclination))*sind(90+PA))+center[1]
        z2[r][theta]=float(f(x,y))*((rr/r0)**times)
        r+=1
    theta=271
    while theta<360:
        r=1
        while r<final_radius:
            rr=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)
            x=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)*cosd(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+PA)+center[0]
            y=(r*((sind(theta)*cosd(inclination))**2+(cosd(theta))**2)**0.5)*sind(np.arctan(tand(theta)*cosd(inclination))*180/math.pi+PA)+center[1]
            z2[r][theta]=float(f(x,y))*((rr/r0)**times)
            r+=1
        theta+=1


    r=0
    theta=0
    while theta<360:
        z2[r][theta]=float(f(center[0],center[1]))
        theta+=1
    plt.imshow(z2, cmap= 'gray',origin='lower')
    return z2

In [ ]:
def gs(x,A1,sigma1,miu1,A0):#gaussian distribution function
    return A1*(1/((2**0.5)*sigma1))*math.e**((-(x-miu1)**2)/(2*sigma1**2))+A0

In [7]:
def fit_for_center(img_p,x_init,x_fin,y_init,y_fin,width=10,mini=None,
                   theta=None,rad=None,prob=None):
    if theta==None & rad==None & prob==None:
        theta=[]
        rad=[]
        prob=[]
    elif theta==None or rad==None or prob==None:
        raise ValueError('please input theta, radius and probability together')
        
    j=y_init
    while (j <=y_fin):
        X1=[]
        Y1=[]
        i=x_init
        max1=0
        a=0
        while i < x_fin:
            if img_p[i][j]>=0:
                y=img_p[i][j]
                X1.append(i)
                Y1.append(y)
                if img_p[i][j]>max1:
                    max1=img_p[i][j]
                    a=i          #a is the location of the maximum 
            i+=1
        xdata=X1
        ydata=Y1
        b=img_p[a][j]
        c=w/2          
        d1=int(min(X1))             
        cen=Y1.index(max(Y1))
        w=width
        popt, pcov = curve_fit(gs,xdata[cen-w:cen+w],ydata[cen-w:cen+w],[b-d1,c,a,d1],maxfev=5000000)

        if (popt[2] < max(X1))&(popt[2] > min(X1)):
            theta.append(popt[2])
            rad.append(j)
            p=np.sqrt(pcov[2][2])
            prob.append(p)
        j+=1 
        return theta, rad, prob
    

In [ ]:
def f_withRn(X,rp,n,a0,a1,a2,a3,a4,a5):
    z,y,k=X
    x=z+y*rp*(k/60)**n
    return a0+a1*x**1+a2*(x)**2+a3*(x)**3+a4*x**4+a5*x**5

def f_withoutRn(X,rp,n,a0,a1,a2,a3,a4,a5):
    z,y=X
    x=z+y*rp
    return a0+a1*x**1+a2*(x)**2+a3*(x)**3+a4*x**4+a5*x**5

In [ ]:
def fitting(theta1,theta2,rad1,rad2,prob1,prob2,est=-0.1,withRn=True):
    
    theta=np.hstack((theta1,theta2))
    theta1b=np.zeros(len(theta1))
    theta2b=np.ones(len(theta2))
    
    thetab=np.hstack((theta1b,theta2b))
    rad=np.hstack((rad1,rad2))
    P=np.hstack((prob1,prob2))

    if withRn==True:
        theta=np.vstack((theta,thetab,rad))
        popt, pcov = curve_fit(f_withRn,thetab,rad,[est]*7,sigma=P,absolute_sigma=True,maxfev=5000000)

    else:
        theta=np.vstack((theta,thetab))
        popt, pcov = curve_fit(f_withoutRn,thetab,rad,[est]*6,sigma=P,absolute_sigma=True,maxfev=5000000)
    
    return popt, pcov
